In [1]:
# ## Inicializando Google Earth Engine
import numpy as np ## For array handling
import pandas as pd ## For dataframes
import ee # # Authenticates and initializes Earth Engine
import geemap # A dynamic module of Google Earth Engine
import fiona # For geopackage file handling 
from shapely.geometry import shape ## Shapes for geometries
import os #To set and modify filepaths
import json # For json objects
import urllib # For download geopackage files
import matplotlib.pyplot as plt
from scipy import signal ## Aplying Savistky Golay
import rasterio as rio

In [2]:
def ee_initialize(): ## Initializing in Google Earth Engine as function
    try:
        ee.Initialize()
    except:
        ee.Authenticate()
        ee.Initialize()
ee_initialize()

In [3]:
def is_url(url): # URL Input verification
    """Check to see if *url* has a valid protocol."""
    import urllib 
    try:
        return urllib.parse.urlparse(url).scheme in ('http', 'https') 
    except Exception:
        return False

In [4]:
def gpkg_to_geojson(in_gpkg, layer= None, out_json=None):
    """Converts specific layer from geopackage file to GeoJSON.
    Args:
        in_gpkg (str): File path or URL of geopackage file.
        layer (str, optional): Layer name or number of the geopackage file. Defaults first layer as None
        out_json (str, optional): File path of the output GeoJSON. Defaults to None.
    Returns:
        object: The json object representing the geopackage layer.
    """   
    ee_initialize()
    try: 
        import fiona
        import json  
        if os.path.exists(in_gpkg): # If the path is a filepath      
                path_or_bytes = os.path.abspath(in_gpkg)
                reader = fiona.open
                if out_json is None:  ## Obtaining empty output json
                    out_json = os.path.splitext(in_gpkg)[0] + ".json"
                elif os.path.exists(out_json): # If the out_json is duplicated
                    out_json = out_json.replace('.json', '_bk.json')  
                elif not os.path.exists(os.path.dirname(out_json)): # If the filepath has not been created yet
                    os.makedirs(os.path.dirname(out_json))    
      
                    
        elif is_url(in_gpkg): # If the path is a URL                  
                path_or_bytes = urllib.request.urlopen(in_gpkg).read()
                reader = fiona.BytesCollection            
                if out_json is None: # If the ouput name of the json is not specified
                    out_json = os.path.split(in_gpkg)[1].split(".")[0] + ".json"
                elif os.path.exists(out_json): # If the out_json is duplicated
                    out_json = out_json.replace('.json', '_bk.json')    
                elif not os.path.exists(os.path.dirname(out_json)): # If the filepath has not been created yet
                    os.makedirs(os.path.dirname(out_json))             

        buffer=[]         
        with reader(path_or_bytes, layer = layer, enabled_drivers="GPKG") as features:
            for feature in features: #Reading each feature of geopackage we obtain a dict keys for json
                    ids = feature["id"]
                    atr = feature["properties"]
                    geom = feature["geometry"]
                    buffer.append(dict(type="Feature", id = ids, geometry=geom, properties=atr)) 

        with open(out_json, "w") as geojson: 
            geojson.write(json.dumps({"type": "FeatureCollection", #Writing in a json our buffer list
                                 "features":buffer}, indent=2))
            geojson.close()          
        with open(out_json) as f:
             json_data = json.load(f) #Reading a full json and return it as result
       
        return json_data

    except Exception as e:
        print(e)

In [5]:
def roi(in_dir):
    with fiona.open(in_dir, "r", enabled_drivers="GPKG") as source:
        for f in source:
            geom = shape(f['geometry']).bounds
        return [[geom[0], geom[3]],
                        [geom[2], geom[3]], 
                        [geom[2],geom[1]], 
                        [geom[0],geom[1]]]

In [6]:
def gpkg_to_ee(in_gpkg, layer=None):
    """Converts specific layer from geopackage file to  Earth Engine object.
    Args:
        in_gpkg (str): File path or URL of geopackage file.
        layer (str, optional): Layer name or number of the geopackage. Defaults first layer as None
        
    Returns:
        object: Earth Engine objects representing the geopackage layer.
    """
    ee_initialize()
    try:
        json_data = gpkg_to_geojson(in_gpkg, layer= layer) ## Converting geopackage file to geojson
        ee_object  = geemap.geojson_to_ee(json_data) ## Converting geojson to ee object
        return ee_object
    except Exception as e:
        print(e) 

In [ ]:
# ##  Descargando coleccion de imagenes MODIS13Q1

In [7]:
## Vector data from filepath
workspace = r"/home/jairflores/Documentos/UNMSM/Fires_Project/AnalisOcurrenIncendiosR/Materiales"
geegpkg = os.path.join(workspace,"Cutervo.gpkg")

## Creating a ee.FeatureCollection
cajamarca_ee= gpkg_to_ee(geegpkg)
roi = roi(geegpkg)

In [8]:
Map= geemap.Map(center=[-12,-75],zoom=5)
Map.add_basemap('SATELLITE')

In [9]:
ee.ImageCollection("MODIS/006/MOD13Q1").aggregate_array('system:band_names').getInfo()[0]

['NDVI',
 'EVI',
 'DetailedQA',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b07',
 'ViewZenith',
 'SolarZenith',
 'RelativeAzimuth',
 'DayOfYear',
 'SummaryQA']

In [10]:
def binarizar(decimal):
    binario = ''
    while decimal // 2 != 0:
        binario = str(decimal % 2) + binario
        decimal = decimal // 2
    return "0"*(16-(len(binario)+1))+ str(decimal) + binario

In [11]:
bin_data =[]
for i in list(range(1,65536)):
           bin_data.append(binarizar(i))

In [12]:
d = {'bin': bin_data, 'decimal': list(range(1,65536))}
data_frame = pd.DataFrame(data=d)
data_frame

,bin,decimal
0,0000000000000001,1
1,0000000000000010,2
2,0000000000000011,3
3,0000000000000100,4
4,0000000000000101,5
...,...,...
65530,1111111111111011,65531
65531,1111111111111100,65532
65532,1111111111111101,65533
65533,1111111111111110,65534


In [13]:
data_bindf = data_frame[data_frame['bin'].apply(lambda x:
                                                 x[2:5] == "011" or x[2:5] == "000" or  x[2:5] == "100" 
                                                 or x[2:5] == "110"  or  x[2:5] == "111" or  x[1] == "1" 
                                                 or  x[8:9] == "00" or x[14:15] == "11" or  x[14:15] == "10")]
data_bindf.head(6)

,bin,decimal
0,0000000000000001,1
1,0000000000000010,2
2,0000000000000011,3
3,0000000000000100,4
4,0000000000000101,5
5,0000000000000110,6


In [15]:
aoi = ee.Geometry.Polygon(roi, None, False)
def scale_factor(image):
# scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start','system:time_end'])
def clipped(image):
    return image.clip(aoi)

In [15]:
ndvi_dataset= ee.ImageCollection("MODIS/006/MOD13Q1").select('NDVI')\
                                 .filter(ee.Filter.date('2000-01-01', '2018-12-31'))\
                                 .map(scale_factor)
qa_dataset = ee.ImageCollection("MODIS/006/MOD13Q1").select('DetailedQA')\
                                 .filter(ee.Filter.date('2000-01-01', '2018-12-31'))
count = int(ndvi_dataset.size().getInfo())
ndvi_images = ndvi_dataset.toList(count)  
qa_images = qa_dataset.toList(count) 

In [16]:
def writeRaster(arr,roi, out_dir):
        new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                                    height = arr.shape[0], width = arr.shape[1],
                                    count=1, dtype=str(arr.dtype),
                                    crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                                    transform=rio.transform.from_bounds(roi[0][0], roi[2][1],roi[2][0], roi[0][1], arr.shape[1], arr.shape[0]))
        new_dataset.write(arr, 1)
        new_dataset.close()

In [17]:
for i in range(0, count):
        ## Obteniendo array a partir del Google Earth Engine
        ndvi_array=np.array(ee.Image(ndvi_images.get(i))\
                                               .sampleRectangle(region=aoi, defaultValue=-2)\
                                               .get("NDVI").getInfo())
        qa_array = np.array(ee.Image(qa_images.get(i))\
                                              .sampleRectangle(region=aoi, defaultValue=0)\
                                              .get("DetailedQA").getInfo())
        
        ##  Cambiar los valores por defecto a NaN values
        ndvi_arr= np.where(ndvi_array==-2,np.nan,ndvi_array)
        qa_arr = np.where(ndvi_array== 0,np.nan,qa_array)
        
        ##  Filtrando el NDVI sin contar los pixeles malos
        mask = np.isin(qa_arr, np.array(data_bindf.decimal))
        qa_arr[mask] = np.nan
        qa_arr[~mask] = 1
        ndvi_filter = ndvi_arr*qa_arr
        
        ## Guardandolo como raster
        start = ee.Date(ee.Image(ndvi_images.get(i)).get("system:time_start")).format('YYYY-MM-dd').getInfo() 
        end = ee.Date(  ee.Image(ndvi_images.get(i)).get("system:time_end")).format('YYYY-MM-dd').getInfo()
        filename= "MOD13Q1_NDVI_filter_250m_16days" + "_" + "{}".format(start) +"_" +"{}".format(end) + ".tif"
        filepath = "/home/jairflores/Documentos/UNMSM/Teledeteccion/MOD13Q1/NDVI/Filtrado"
        out_dir = os.path.join(filepath,filename)
        writeRaster(ndvi_filter,roi,out_dir)

In [16]:
import rasterio
from rasterio.enums import Resampling
def rescaling(in_dir,out_dir, scale):
    
        with rasterio.open(in_dir) as dataset:
            
            # resample data to target shape
            data =dataset.read( 
                out_shape=(
                    dataset.count,
                    int(dataset.height * scale),
                    int(dataset.width * scale)
                ),
                resampling=Resampling.bilinear
            )

            # scale image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )
          
        new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                                    height = int(dataset.height * scale) , width =  int(dataset.width * scale),
                                    count=1, dtype=str(data.dtype),
                                    crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                                    transform = dataset.transform * dataset.transform.scale(
                                    (dataset.width / data.shape[-1]),
                                    (dataset.height / data.shape[-2]) ))
        new_dataset.write(data)
        new_dataset.close()

In [18]:
filepath = "/home/jairflores/Documentos/UNMSM/Teledeteccion/MOD13Q1/NDVI/Filtrado"
filepath_res = "/home/jairflores/Documentos/UNMSM/Teledeteccion/MOD13Q1/NDVI/Resampleado"
for t in os.listdir(filepath):
        filename= "MOD13Q1_NDVI_resamp_5km_16days" + "_" + "{}".format(t.split("_")[5]) +"_" +"{}".format(t.split("_")[6])
        out_dir = os.path.join(filepath_res,filename)
        rescaling(os.path.join(filepath, t),out_dir, scale=1/20)

In [76]:
def fixed_index(index):
    fixed = []
    for a in range(0,len(index)):
        mean = np.nanmean(index,axis=0)
        fixed.append(np.where(np.isnan(index[a]),mean,index[a]))
    return fixed

In [90]:
index=[]
date = []
for t in os.listdir(filepath_res):
        with rio.open(os.path.join(filepath_res,t)) as src:
                    index.append(src.read(1))        
                    transform_res = src.transform
                    date.append(t.split("_")[5])
ind_ = fixed_index(index)                  
# Fill NA index data
# Dictionary
data = {"Date":date,"Index":ind_}
# Data frame
df = pd.DataFrame(data,columns = ["Date","Index"])
# Converting to date time
df["Date"] =pd.to_datetime(df["Date"], format='%Y-%m-%d', errors='coerce')
df.sort_values(by="Date", key=pd.to_datetime, inplace=True)
df
# Interpolate missing data in time series
df["Index"] = pd.Series(df["Index"]).interpolate(limit = len(date))

## Interpolando los valores a través del filtro Savistky Golaydf
df["SavGol"]=df["Index"].apply(lambda x: signal.savgol_filter(x,7,4))

In [107]:
filepath_savgol = "/home/jairflores/Documentos/UNMSM/Teledeteccion/MOD13Q1/NDVI/SavGol"
for i in range(0,len(df["SavGol"])):
    filename= "MOD13Q1_NDVI_SavGol_5km_16days" + "_" + "{}".format(str(df["Date"][i]).split(" ")[0]) + ".tif"
    out_dir = os.path.join(filepath_savgol,filename)
    arr=df["SavGol"][i]
    new_dataset = rio.open(out_dir, 'w', driver='GTiff',
                            height = arr.shape[0], width = arr.shape[1],
                            count=1, dtype=str(arr.dtype),
                            crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ',
                            transform= transform_res)
    new_dataset.write(arr, 1)
    new_dataset.close()

In [135]:
filepath_savgol = "/home/jairflores/Documentos/UNMSM/Teledeteccion/MOD13Q1/NDVI/SavGol"
tif_list=[]
for i in os.listdir(filepath_savgol):
       tif_list.append(os.path.join(filepath_savgol,i))
# Read metadata of first file
with rasterio.open(tif_list[0]) as src0:
    meta = src0.meta
# Update meta to reflect the number of layers
meta.update(count = len(tif_list))
# Read each layer and write it to stack
with rasterio.open(os.path.join(filepath_savgol,'stack.tif'), 'w', **meta) as dst:
    for id, layer in enumerate(tif_list, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [ ]:
ndvi_month = df.groupby(by="month")["SavGol"].apply(np.mean)
ndvi_month

In [138]:
df['month'] = pd.DatetimeIndex(df['Date']).month
df.to_csv(os.path.join(filepath_savgol,"data_SavGol.csv"))